# Bag Of Word Redo

- Viết chương trình dùng thư viện Numpy và các hàm build in của Python để truy vấn văn bản dùng kỹ thuật Bag of Word

## Input:
    - Tập văn bản (corpus)
    - Văn bản test (test doc)
    - N: số lượng văn bản cần trả lại (= 5)
    - Stop_words để loại bỏ các từ không cần thiết

## Output:
    - Top-1 văn bản giống nhất với test doc trong 3 TH:
        + Dùng khoảng cách cosine
        + Dùng khoảng cách euclid
        + Dùng khoảng cách Manhattan

## Result:
    - Hit nếu trả về doc trong top 5 docs của Querry
    - Mis

# Data: 
    - data.txt sử dụng để xây dựng corpus (docID: .I với nội dung: .W)
    - query.txt sử dụng để test docs (queryID: .I với nội dung .W)
    - rel.txt sử dụng nhãn test (queryID: cột 1 với danh sách của docs liên quan: cột 2)

# Yêu cầu:
    - Báo cáo độ chính xác của 3 hàm KC và phân tích kết quả thực thi
    - Accuracy = total # của Hit array giữa tất cả các Querries
    - So sánh độ chính xác dùng và không dùng của loại bỏ và không loại bỏ stop Words

# Khoảng cách Cosine
    - Quan tâm đến hướng
    - Ứng dụng: Tìm kiếm, so sánh, ...
# Khoảng cách Euclid
    - Quan tâm đến độ dài
    - Ứng dụng: Tính toán số học, ...

In [42]:
import numpy as np

# Hàm trích xuất ID + văn bản

In [43]:
def extract_W_sections(filename, capacity="all"):
    """
    Trích xuất các phần .I và .W từ file data
    
    Args:
        filename: Tên file cần đọc
        capacity: Số lượng văn bản cần trả về ("all" hoặc số nguyên)
    
    Returns:
        List ID của các văn bản .I 
        List các văn bản tương ứng .W
    """
    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()
    
    documents = []
    current_doc = {}
    current_section = None
    current_text = []
    
    for line in content.split('\n'):
        line = line.strip()
        
        if line.startswith('.I'):
            # Lưu document trước (nếu có)
            if current_section == 'W' and current_text:
                current_doc['W'] = ' '.join(current_text)
            if current_doc:
                documents.append(current_doc)
            
            # Bắt đầu document mới và lưu ID
            doc_id = line.split()[1] if len(line.split()) > 1 else ''
            current_doc = {'I': doc_id}
            current_text = []
            current_section = None
            
        elif line.startswith('.W'):
            # Bắt đầu phần văn bản
            if current_section == 'W' and current_text:
                current_doc['W'] = ' '.join(current_text)
            current_section = 'W'
            current_text = []
            
        elif line.startswith('.'):
            # Các section khác (.T, .A, .X)
            if current_section == 'W' and current_text:
                current_doc['W'] = ' '.join(current_text)
                current_text = []
            current_section = line[1]
            
        elif current_section == 'W' and line:
            # Nội dung của section .W
            current_text.append(line)
    
    # Xử lý document cuối cùng
    if current_section == 'W' and current_text:
        current_doc['W'] = ' '.join(current_text)
    if current_doc:
        documents.append(current_doc)
    
    # Trả về ID và văn bản .W
    i_sections = [doc.get('I', '') for doc in documents if 'I' in doc]
    w_sections = [doc.get('W', '') for doc in documents if 'W' in doc]
    
    # Xử lý capacity
    if capacity == "all":
        return i_sections, w_sections
    else:
        return i_sections[:capacity], w_sections[:capacity]

# Sử dụng - SỬA LỖI
corpus_ids, corpus_texts = extract_W_sections('data.txt')  # Trả về tuple (IDs, texts)
print(f"Số lượng văn bản: {len(corpus_texts)} documents")

Số lượng văn bản: 1460 documents


# Xây dựng StopWord

In [44]:
def build_stopWords(method='frequency', corpus=None, top_percent=0.05, min_freq=None):
    """
    Xây dựng danh sách stopwords
    
    Args:
        method: 'predefined' (dùng list có sẵn) hoặc 'frequency' (dựa trên tần suất)
        corpus: Danh sách văn bản để phân tích tần suất
        top_percent: % từ phổ biến nhất cần loại bỏ (mặc định 5%)
        min_freq: Ngưỡng tần suất tối thiểu (tùy chọn)
    
    Returns:
        Set các stopwords
    """
    
    if method == 'predefined':
        # Danh sách stopwords tiếng Anh cơ bản
        stopwords = {
            'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from',
            'has', 'he', 'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the',
            'to', 'was', 'will', 'with', 'the', 'this', 'but', 'they', 'have',
            'had', 'what', 'when', 'where', 'who', 'which', 'why', 'how',
            'all', 'each', 'every', 'both', 'few', 'more', 'most', 'other',
            'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
            'than', 'too', 'very', 'can', 'will', 'just', 'should', 'now'
        }
        return stopwords
    
    elif method == 'frequency':
        if corpus is None:
            raise ValueError("Corpus không được để trống khi dùng method='frequency'")
        
        # Đếm tần suất xuất hiện của từng từ
        word_freq = {}
        total_words = 0
        
        for doc in corpus:
            words = doc.lower().split()
            for word in words:
                word_freq[word] = word_freq.get(word, 0) + 1
                total_words += 1
        
        # Sắp xếp theo tần suất giảm dần
        sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
        
        # Lấy top % từ phổ biến nhất
        num_stopwords = int(len(sorted_words) * top_percent)
        stopwords = set([word for word, freq in sorted_words[:num_stopwords]])
        
        # Nếu có min_freq, thêm điều kiện lọc
        if min_freq:
            stopwords = {word for word, freq in sorted_words 
                        if freq >= min_freq and word in stopwords}
        
        return stopwords
    
    elif method == 'combined':
        # Kết hợp cả hai: bắt đầu với predefined, bổ sung từ frequency
        stopwords = build_stopWords(method='predefined')
        
        if corpus:
            freq_stopwords = build_stopWords(method='frequency', corpus=corpus, 
                                            top_percent=top_percent, min_freq=min_freq)
            stopwords = stopwords.union(freq_stopwords)
        
        return stopwords
    
    else:
        raise ValueError("Method phải là 'predefined', 'frequency', hoặc 'combined'")
    

# Kiểm tra với None stopWords trước
stop_words = []

# Hàm xử lí từ và xây dựng hệ thống BoW

In [45]:
def build_vocab(corpus, stop_words):
    """
    Xây dựng hệ thống từ điển từ corpus, loại bỏ stop words.
    """
    
    word_list = []
    for sentence in corpus:
        words = sentence.lower().split()
        words = [w for w in words if w not in stop_words]
        word_list += words

    dictionary = sorted(set(word_list))
    return dictionary

def vectorize_sentence(text, dictionary, stop_words):
    """
    Biểu diễn câu dưới dạng vector (Bag of Words).
    ⚡ TỐI ƯU: Dùng dict đếm từ thay vì list.count() để tăng tốc độ
    """
    words = text.lower().split()
    words = [w for w in words if w not in stop_words]
    
    # Tạo dict đếm từ (thay vì dùng list.count() chậm)
    word_count = {}
    for word in words:
        word_count[word] = word_count.get(word, 0) + 1
    
    # Tạo vector dựa trên dictionary
    vector = np.zeros(len(dictionary), dtype=int)
    for i, word in enumerate(dictionary):
        vector[i] = word_count.get(word, 0)  # Lấy count, mặc định 0 nếu không có
    
    return vector

def corpus_to_matrix(corpus, dictionary, stop_words):
    """
    Chuyển toàn bộ corpus thành ma trận BoW
    """
    matrix = []
    for doc in corpus:
        vector = vectorize_sentence(doc, dictionary, stop_words)
        matrix.append(vector)
    
    return np.array(matrix)

# Các hàm tính khoảng cách

In [46]:
def euclidean_distance(a, b):
    """
    Tính khoảng cách Euclid
    """
    return np.sqrt(np.sum((np.array(a) - np.array(b)) ** 2))

def cosine_distance(a, b):
    """
    Tính khoảng cách Cosine
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    # Tránh chia cho 0
    if norm_a == 0 or norm_b == 0:
        return 1.0
    
    return 1 - dot_product / (norm_a * norm_b)

def manhattan_distance(a, b):
    """
    Tính khoảng cách Manhattan
    """
    return np.sum(np.abs(np.array(a) - np.array(b)))

# Hàm truy vấn top-N văn bản giống test_doc nhất

In [47]:
def retrieve_documents(corpus, test_doc, n, stop_words, method='cosine', 
                       dictionary=None, corpus_matrix=None):
    """
    Truy vấn top-N văn bản giống test_doc nhất
    
    Args:
        corpus: Tập văn bản
        test_doc: Văn bản cần tìm kiếm
        n: Số lượng văn bản trả về
        stop_words: Danh sách stop words
        method: 'cosine', 'euclidean', hoặc 'manhattan'
        dictionary: (Optional) Từ điển đã build sẵn để tăng tốc
        corpus_matrix: (Optional) Ma trận corpus đã convert sẵn để tăng tốc
    
    Returns:
        List các tuple (index, distance, document)
    """
    # ⚡ TỐI ƯU: Dùng dictionary & corpus_matrix có sẵn thay vì build lại
    if dictionary is None:
        dictionary = build_vocab(corpus, stop_words)
    
    if corpus_matrix is None:
        corpus_matrix = corpus_to_matrix(corpus, dictionary, stop_words)
    
    # Chuyển test_doc thành vector
    test_vector = vectorize_sentence(test_doc, dictionary, stop_words)
    
    # Tính khoảng cách
    distances = []
    for i, doc_vector in enumerate(corpus_matrix):
        if method == 'cosine':
            dist = cosine_distance(test_vector, doc_vector)
        elif method == 'euclidean':
            dist = euclidean_distance(test_vector, doc_vector)
        elif method == 'manhattan':
            dist = manhattan_distance(test_vector, doc_vector)
        else:
            raise ValueError("Method phải là 'cosine', 'euclidean', hoặc 'manhattan'")
        
        distances.append((i, dist, corpus[i]))
    
    # Sắp xếp theo khoảng cách (tăng dần)
    distances.sort(key=lambda x: x[1])
    
    # Trả về top-N
    return distances[:n]

# DEMO

In [48]:
#print("\nCorpus:")
#for i, doc in enumerate(corpus[:3]):
#    print(f"  [{i}] {doc[:100]}...")

# Văn bản test
corpus_ids, corpus_texts = extract_W_sections('data.txt')  # Lấy tất cả
corpus = corpus_texts  # Chỉ lấy phần văn bản
print(f"✓ Số lượng văn bản (all): {len(corpus)} documents")
print(f"  - IDs: {len(corpus_ids)}")
print(f"  - Texts: {len(corpus_texts)}")

# ⚡ TỐI ƯU: Xây dựng từ điển và corpus matrix MỘT LẦN
print(f"\n⚡ Pre-computing dictionary và corpus matrix...")
dictionary = build_vocab(corpus, stop_words)
corpus_matrix = corpus_to_matrix(corpus, dictionary, stop_words)
print(f"✓ Dictionary: {len(dictionary)} từ")
print(f"✓ Corpus matrix: {corpus_matrix.shape}")

n = 1  # Số lượng văn bản cần trả về

✓ Số lượng văn bản (all): 1460 documents
  - IDs: 1460
  - Texts: 1460

⚡ Pre-computing dictionary và corpus matrix...
✓ Dictionary: 17441 từ
✓ Corpus matrix: (1460, 17441)


# Truy Vấn

In [49]:
print("KẾT QUẢ TRUY VẤN")

# Lấy ID và văn bản từ query
query_ids, query_texts = extract_W_sections('querry.txt')

# Truy vấn với 3 phương pháp
cosine_results = {}
euclidean_results = {}
manhattan_results = {}

# ⚡ Truy vấn với Cosine Distance (dùng corpus_matrix có sẵn)
print(f"\nTop-{n} văn bản (Cosine Distance):")
for query_id, query_text in zip(query_ids, query_texts):
    results = retrieve_documents(corpus, query_text, n, stop_words, method='cosine',
                                 dictionary=dictionary, corpus_matrix=corpus_matrix)
    cosine_results[query_id] = results[0][0]  # Lưu ID của document trả về
    
    for rank, (idx, dist, doc) in enumerate(results, 1):
        print(f"  Query {query_id} - {rank}. Doc[{idx}] Distance: {dist:.4f}")

# ⚡ Truy vấn với Euclidean Distance (dùng corpus_matrix có sẵn)
print(f"\nTop-{n} văn bản (Euclidean Distance):")
for query_id, query_text in zip(query_ids, query_texts):
    results = retrieve_documents(corpus, query_text, n, stop_words, method='euclidean',
                                 dictionary=dictionary, corpus_matrix=corpus_matrix)
    euclidean_results[query_id] = results[0][0]
    
    for rank, (idx, dist, doc) in enumerate(results, 1):
        print(f"  Query {query_id} - {rank}. Doc[{idx}] Distance: {dist:.4f}")

# ⚡ Truy vấn với Manhattan Distance (dùng corpus_matrix có sẵn)
print(f"\nTop-{n} văn bản (Manhattan Distance):")
for query_id, query_text in zip(query_ids, query_texts):
    results = retrieve_documents(corpus, query_text, n, stop_words, method='manhattan',
                                 dictionary=dictionary, corpus_matrix=corpus_matrix)
    manhattan_results[query_id] = results[0][0]
    
    for rank, (idx, dist, doc) in enumerate(results, 1):
        print(f"  Query {query_id} - {rank}. Doc[{idx}] Distance: {dist:.4f}")

KẾT QUẢ TRUY VẤN

Top-1 văn bản (Cosine Distance):
  Query 1 - 1. Doc[16] Distance: 0.4495
  Query 2 - 1. Doc[1265] Distance: 0.6237
  Query 3 - 1. Doc[1165] Distance: 0.7348
  Query 4 - 1. Doc[1189] Distance: 0.6488
  Query 5 - 1. Doc[374] Distance: 0.5671
  Query 6 - 1. Doc[626] Distance: 0.6003
  Query 7 - 1. Doc[923] Distance: 0.4849
  Query 8 - 1. Doc[132] Distance: 0.5082
  Query 9 - 1. Doc[1111] Distance: 0.5127
  Query 10 - 1. Doc[1] Distance: 0.5122
  Query 11 - 1. Doc[534] Distance: 0.4443
  Query 12 - 1. Doc[1189] Distance: 0.5273
  Query 13 - 1. Doc[146] Distance: 0.4756
  Query 14 - 1. Doc[1143] Distance: 0.7275
  Query 15 - 1. Doc[663] Distance: 0.4805
  Query 16 - 1. Doc[59] Distance: 0.4488
  Query 17 - 1. Doc[1157] Distance: 0.6628
  Query 18 - 1. Doc[807] Distance: 0.6100
  Query 19 - 1. Doc[816] Distance: 0.5944
  Query 20 - 1. Doc[1165] Distance: 0.7500
  Query 21 - 1. Doc[647] Distance: 0.4334
  Query 22 - 1. Doc[132] Distance: 0.5138
  Query 23 - 1. Doc[201] Dista

# Hàm so sánh độ chính xác với thực tế

In [50]:
def extract_results():
    """
    Đọc file result.txt và trích xuất query ID và relevant doc IDs
    Returns:
        Dictionary: {query_id: [list of relevant doc ids]}
    """
    relevant_docs = {}
    with open('result.txt', 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split()
                if len(parts) >= 2:
                    query_id = parts[0]
                    doc_id = parts[1]
                    
                    if query_id not in relevant_docs:
                        relevant_docs[query_id] = []
                    relevant_docs[query_id].append(doc_id)
    
    return relevant_docs

def compare_methods(cosine_results=cosine_results, euclidean_results=euclidean_results, manhattan_results=manhattan_results):
    """
    So sánh kết quả từ 3 phương pháp với ground truth
    Trả về accuracy (%)
    """
    # Lấy ground truth từ result.txt
    relevant_docs = extract_results()
    
    # Tính accuracy cho từng phương pháp
    methods = {
        'cosine': cosine_results,
        'euclidean': euclidean_results,
        'manhattan': manhattan_results
    }
    
    results = {}
    
    for method_name, method_results in methods.items():
        hits = 0
        total = 0
        
        for query_id, retrieved_doc_id in method_results.items():
            total += 1
            # Kiểm tra xem retrieved_doc_id có trong danh sách relevant docs không
            if query_id in relevant_docs:
                if str(retrieved_doc_id) in relevant_docs[query_id]:
                    hits += 1
        
        # Tính tỉ lệ %
        accuracy = (hits / total * 100) if total > 0 else 0
        results[method_name] = {
            'hits': hits,
            'total': total,
            'accuracy': accuracy
        }
        
        print(f"\n{method_name.upper()}:")
        print(f"  Hits: {hits}/{total}")
        print(f"  Accuracy: {accuracy:.2f}%")
    
    return results

# Gọi hàm so sánh
compare_results = compare_methods(cosine_results, euclidean_results, manhattan_results)


COSINE:
  Hits: 4/112
  Accuracy: 3.57%

EUCLIDEAN:
  Hits: 3/112
  Accuracy: 2.68%

MANHATTAN:
  Hits: 3/112
  Accuracy: 2.68%


# Truy vấn lại với StopWords

In [51]:
stop_words = list(build_stopWords(method='combined', corpus=corpus_texts, top_percent=0.05))

print("KẾT QUẢ TRUY VẤN CÓ STOP WORDS")

# Lấy ID và văn bản từ query
query_ids, query_texts = extract_W_sections('querry.txt')

# Truy vấn với 3 phương pháp
cosine_results_with_stopwords = {}
euclidean_results_with_stopwords = {}
manhattan_results_with_stopwords = {}

# Truy vấn với Cosine Distance
print(f"\nTop-{n} văn bản (Cosine Distance):")
for query_id, query_text in zip(query_ids, query_texts):
    results = retrieve_documents(corpus, query_text, n, stop_words, method='cosine')
    cosine_results_with_stopwords[query_id] = results[0][0]  # Lưu ID của document trả về
    
    for rank, (idx, dist, doc) in enumerate(results, 1):
        print(f"  Query {query_id} - {rank}. Doc[{idx}] Distance: {dist:.4f}")

# Truy vấn với Euclidean Distance
print(f"\nTop-{n} văn bản (Euclidean Distance):")
for query_id, query_text in zip(query_ids, query_texts):
    results = retrieve_documents(corpus, query_text, n, stop_words, method='euclidean')
    euclidean_results_with_stopwords[query_id] = results[0][0]
    
    for rank, (idx, dist, doc) in enumerate(results, 1):
        print(f"  Query {query_id} - {rank}. Doc[{idx}] Distance: {dist:.4f}")

# Truy vấn với Manhattan Distance
print(f"\nTop-{n} văn bản (Manhattan Distance):")
for query_id, query_text in zip(query_ids, query_texts):
    results = retrieve_documents(corpus, query_text, n, stop_words, method='manhattan')
    manhattan_results_with_stopwords[query_id] = results[0][0]
    
    for rank, (idx, dist, doc) in enumerate(results, 1):
        print(f"  Query {query_id} - {rank}. Doc[{idx}] Distance: {dist:.4f}")

KẾT QUẢ TRUY VẤN CÓ STOP WORDS

Top-1 văn bản (Cosine Distance):
  Query 1 - 1. Doc[466] Distance: 0.8586
  Query 2 - 1. Doc[596] Distance: 0.8639
  Query 3 - 1. Doc[468] Distance: 0.7278
  Query 4 - 1. Doc[35] Distance: 0.7818
  Query 5 - 1. Doc[861] Distance: 0.7675
  Query 6 - 1. Doc[1132] Distance: 0.7418
  Query 7 - 1. Doc[151] Distance: 0.9000
  Query 8 - 1. Doc[538] Distance: 0.6220
  Query 9 - 1. Doc[1132] Distance: 0.7764
  Query 10 - 1. Doc[564] Distance: 0.6362
  Query 11 - 1. Doc[133] Distance: 0.6797
  Query 12 - 1. Doc[1209] Distance: 0.7990
  Query 13 - 1. Doc[363] Distance: 0.8492
  Query 14 - 1. Doc[0] Distance: 1.0000
  Query 15 - 1. Doc[655] Distance: 0.7396
  Query 16 - 1. Doc[733] Distance: 0.8399
  Query 17 - 1. Doc[36] Distance: 0.8709
  Query 18 - 1. Doc[265] Distance: 0.8207
  Query 19 - 1. Doc[178] Distance: 0.6307
  Query 20 - 1. Doc[133] Distance: 0.5196
  Query 21 - 1. Doc[1204] Distance: 0.6464
  Query 22 - 1. Doc[914] Distance: 0.5528
  Query 23 - 1. Doc[

# So sánh độ chính xác với thực tế với StopWords

In [52]:
compare_results_with_stopwords = compare_methods(cosine_results_with_stopwords, euclidean_results_with_stopwords, manhattan_results_with_stopwords)


COSINE:
  Hits: 4/112
  Accuracy: 3.57%

EUCLIDEAN:
  Hits: 1/112
  Accuracy: 0.89%

MANHATTAN:
  Hits: 1/112
  Accuracy: 0.89%
